In [24]:
import numpy as np
import pandas as pd
import scipy.io
import h5py

In [26]:
# downloaded from https://data.mendeley.com/datasets/p7ft2bvphx/1
v1 = scipy.io.loadmat('data/empirical/SPKcounts_V20160929cue_MW_250X250ms.mat')
v2 = scipy.io.loadmat('data/empirical/SPKcounts_V20160930cue_MW_250X250ms.mat')
v3 = scipy.io.loadmat('data/empirical/SPKcounts_V20161005cue_MW_250X250ms.mat')
v4 = scipy.io.loadmat('data/empirical/SPKcounts_V20161017cue_MW_250X250ms.mat')
w1 = scipy.io.loadmat('data/empirical/SPKcounts_W20160112cue_MW_250X250ms.mat')
w2 = scipy.io.loadmat('data/empirical/SPKcounts_W20160113cue_MW_250X250ms.mat')
w3 = scipy.io.loadmat('data/empirical/SPKcounts_W20160121cue_MW_250X250ms.mat')
w4 = scipy.io.loadmat('data/empirical/SPKcounts_W20160122cue_MW_250X250ms.mat')
vs = [v1, v2, v3, v3]
ws = [w1, w2, w3, w4]

In [31]:
dfs = []
columns = ['monkey', 'session', 'block_type', 'trial', 'left', 'right', 'correct', 'reversal_at_trial', 'monkey_accuracy']

for monkey in ['V', 'W']:
    files = vs if monkey=='V' else ws
    for f, file in enumerate(files):
        print(f)
        df = pd.DataFrame(columns=columns)
        data = file['Y']
        bid = data[:,7]
        block = data[:,9]
        trial = data[:,5]
        letter = data[:,0]
        location = data[:,1]
        best_chosen = data[:,4]
        block_completed = data[:,12]
        # reversal_at_trial = data[:,6]
        idxs = np.argwhere(data[:,6]==1)[:,0]
        reversal_at_trial = {}
        for idx in idxs:
            bb = bid[idx]
            tt = trial[idx]
            reversal_at_trial[bb]=tt
        # print(idxs, reversal_at_trial)
        # raise
        # reversal_at_trial = []
        # for bb in np.unique(bid):
        #     idx = np.argwhere(data[:,6]==1)[bb-1][0]
        #     reversal_at_trial.append(trial[idx])
        # print(reversal_at_trial)
        left = []
        right = []
        correct = []
        for row in range(data.shape[0]):
            if location[row]==0 and letter[row]==0:
                left.append('A')
                right.append('B')
            if location[row]==0 and letter[row]==1:
                left.append('B')
                right.append('A')
            if location[row]==1 and letter[row]==0:
                left.append('B')
                right.append('A')
            if location[row]==1 and letter[row]==1:
                left.append('A')
                right.append('B')
            # if block[row] == 2:  # where
            if best_chosen[row]==1 and location[row]==0:
                correct.append('left')
            if best_chosen[row]==1 and location[row]==1:
                correct.append('right')
            if best_chosen[row]==0 and location[row]==0:
                correct.append('right')
            if best_chosen[row]==0 and location[row]==1:
                correct.append('left')
            # if block[row] == 1:  # what
            #     if best_chosen[row]==1 and letter[row]==0:
            #         correct.append('left')
            #     if best_chosen[row]==1 and letter[row]==1:
            #         correct.append('right')
            #     if best_chosen[row]==0 and letter[row]==0:
            #         correct.append('right')
            #     if best_chosen[row]==0 and letter[row]==1:
            #         correct.append('left')
            # print(f"block {block[row]}, left {left[-1]}, right {right[-1]}, chose {best_chosen[row]}, correct {correct[-1]}") 
        df['block'] = bid
        df['block_type'] = block
        df['trial'] = trial
        df['left'] = left
        df['right'] = right
        df['correct'] = correct
        df['monkey_accuracy'] = best_chosen
        df['block_completed'] = block_completed
        df = df[df['block_completed'] != 0]
        temp = []
        for bb in df['block'].unique():
            for t in df.query("block==@bb")['trial'].unique():
                temp.append(reversal_at_trial[bb])
            # print(bid, t)
        df['reversal_at_trial'] = temp
        df['monkey'] = [monkey for _ in range(df.shape[0])]
        df['session'] = [f for _ in range(df.shape[0])]
        dfs.append(df)

behavioral_data = pd.concat(dfs, ignore_index=True)
# behavioral_data['left'] = behavioral_data['letter'].replace('A', -1)
# behavioral_data['left'] = behavioral_data['letter'].replace('B', 1)
# behavioral_data['right'] = behavioral_data['location'].replace('A', -1)
# behavioral_data['right'] = behavioral_data['location'].replace('B', 1)
behavioral_data['block_type'] = behavioral_data['block_type'].replace(1, 'what')
behavioral_data['block_type'] = behavioral_data['block_type'].replace(2, 'where')
# behavioral_data = behavioral_data[behavioral_data['block_completed'] != 0]
behavioral_data = behavioral_data.drop(columns='block_completed')

0
1
2
3
0
1
2
3


In [33]:
behavioral_data.query("monkey=='V' & block==10")[20:60]

,monkey,session,block_type,trial,left,right,correct,reversal_at_trial,monkey_accuracy,block
740,V,0,what,21,A,B,left,50,1,10
741,V,0,what,22,A,B,left,50,1,10
742,V,0,what,23,B,A,right,50,1,10
743,V,0,what,24,B,A,right,50,1,10
744,V,0,what,25,A,B,left,50,1,10
745,V,0,what,26,B,A,right,50,1,10
746,V,0,what,27,B,A,right,50,1,10
747,V,0,what,28,A,B,left,50,1,10
748,V,0,what,29,B,A,right,50,1,10
749,V,0,what,30,A,B,left,50,1,10


In [34]:
behavioral_data.to_pickle("data/empirical.pkl")

In [13]:
def check_opposite(row):
    return (row['left'] == 'A' and row['right'] == 'B') or \
           (row['left'] == 'B' and row['right'] == 'A')

# Apply the check to all rows
all_opposite = behavioral_data.apply(check_opposite, axis=1).all()

print("All rows contain opposite letters:", all_opposite)

All rows contain opposite letters: True
